# Criação e preenchimento das relações na base de dados

Primeiro, importamos a biblioteca "psycopg2", que é utilizada para fazer a conexão do script com a base de dados do PostgreSQL.

In [66]:
import psycopg2
import names
import random as r
import pandas as pd
from datetime import datetime

Criada a database no pgAdmin, possível fazer a conexão.

In [2]:
DB_NAME = "heloo_bi"
DB_USER = "postgres"
DB_PASS = "tayrone250595"
DB_HOST = "localhost"

conn = psycopg2.connect(dbname = DB_NAME, user = DB_USER, password = DB_PASS, host = DB_HOST)

O cursor é o objeto utilizado para executar os comandos SQL.

In [3]:
cursor = conn.cursor()

Primeiro, todas as tabelas são criadas, como sugeridas na descrição do teste.

In [4]:
cursor.execute('''CREATE TABLE medicos(
    id BIGSERIAL NOT NULL PRIMARY KEY,
    crm TEXT,
    nome TEXT,
    especialidade TEXT,
    telefone TEXT
    );''')

cursor.execute('''CREATE TABLE pacientes(
    id BIGSERIAL NOT NULL PRIMARY KEY,
    nome TEXT,
    telefone TEXT
    );''')

cursor.execute('''CREATE TABLE consultas_situacoes(
    id BIGSERIAL NOT NULL PRIMARY KEY,
    nome TEXT NOT NULL
    );''')

cursor.execute('''CREATE TABLE cidades(
    id BIGSERIAL NOT NULL PRIMARY KEY,
    nome TEXT NOT NULL,
    estado_sigla TEXT NOT NULL
    );''')

cursor.execute('''CREATE TABLE consultas(
    id BIGSERIAL NOT NULL PRIMARY KEY,
    paciente_id BIGINT NOT NULL REFERENCES pacientes(id), 
    medico_id BIGINT NOT NULL REFERENCES medicos(id), 
    data_agendamento TIMESTAMP,
    situacao_id BIGINT NOT NULL REFERENCES consultas_situacoes(id), 
    cidade_id BIGINT NOT NULL REFERENCES cidades(id)
    );''')

conn.commit()


Para popular a base de dados, vamos gerar mil entradas para a relação "Médicos", usando dados aleatórios.
Primeiro, geramos os nome, usando a biblioteca "names".

In [5]:
names.get_full_name()

'Michael Blanchard'

Depois, geramos os números de telefone.

In [6]:
def gerar_telefone():
    telefone = []
    
    # O primeiro número é 9
    telefone.append(9)

    # Os dois próximos são 88 ou 99
    telefone.append(r.randint(8, 9))
    telefone.append(telefone[1])

    # O restante são números de 0 a 9

    for i in range(3, 9):
        telefone.append(r.randint(0, 9))
    

    telefone = int(''.join(map(str, telefone)))
    return telefone

gerar_telefone()

999240886

Agora, geramos os CRM.

In [8]:
def gerar_crm():
    crm = []

    crm.append('CRM')

    for i in range(0, 4):
        crm.append(r.randint(0, 9))

    crm.append('/PB')   

    crm = (''.join(map(str, crm)))

    return crm

gerar_crm()

'CRM8860/PB'

Para as especialidades, usando a lista com todas as 55 especialidades existentes do Conselhor Federal de Medicina.

In [9]:
with open("./dados/especialidades.txt") as file:
    especialidades = file.readlines()
    especialidades = [line.rstrip() for line in especialidades]

especialidades[0:5]

['Acupuntura',
 'Alergia e imunologia',
 'Anestesiologia',
 'Angiologia',
 'Cardiologia']

In [10]:
for i in range(0, 1000):
    instancia_medico = (gerar_crm(), names.get_full_name(), r.choice(especialidades), gerar_telefone())

    cursor.execute('INSERT INTO medicos (crm, nome, especialidade, telefone) VALUES (%s, %s, %s, %s)', 
                   instancia_medico)

conn.commit()

Podemos conferir se a tabela agora tem o número esperado de instâncias, com o seguinte comando SQL:
'SELECT count(*) AS exact_count FROM medicos;'

Seguimos preenchendo as outras relações, de forma semelhante. Pacientes:

In [14]:
for i in range(0, 1000):
    instancia_paciente = (names.get_full_name(), gerar_telefone())

    cursor.execute('INSERT INTO pacientes (nome, telefone) VALUES (%s, %s)', 
                   instancia_paciente)

conn.commit()

Situações de consultas:

In [26]:
situacoes = ['Pendente', 'Executada', 'Cancelada']

for i in range(0, 1000):

    instancia_situacao = (r.choice(situacoes), )

    cursor.execute('INSERT INTO consultas_situacoes (nome) VALUES (%s)', 
                   instancia_situacao)

conn.commit()
    

Cidades:

In [44]:
municipios_brasil = pd.read_csv("./dados/municipios_brasil.txt", sep = ";", usecols=['UF', 'Município'])
municipios_brasil[:5]

,UF,Município
0,RO,Alta Floresta D´oeste
1,RO,Ariquemes
2,RO,Cabixi
3,RO,Cacoal
4,RO,Cerejeiras


Foi usada a lista de todos os municípios brasileiros, provida pelo governo federal (http://blog.mds.gov.br/redesuas/lista-de-municipios-brasileiros/).

In [46]:
cidade = municipios_brasil.sample()

cidade['UF'].item()

'SP'

In [56]:
cidade = municipios_brasil.sample()

type(cidade['UF'].item())
    
    #instancia_cidade = (cidade['Município'].item(), cidade['UF'].item())

str

In [64]:
for i in range(0, 1000):

    cidade = municipios_brasil.sample()
        
    instancia_cidade = (cidade['Município'].item(), cidade['UF'].item())
        
    cursor.execute('INSERT INTO cidades (nome, estado_sigla) VALUES (%s, %s)', 
                   instancia_cidade)


conn.commit()

Consultas:

In [71]:
for i in range(0, 1000):
    
    instancia_consulta = (r.randint(1, 1000), r.randint(2, 1000), datetime.today().strftime('%Y-%m-%d'), 
                          r.randint(1, 1000), r.randint(1, 1000))

    cursor.execute('''INSERT INTO consultas (paciente_id, medico_id, data_agendamento, situacao_id, cidade_id) 
                   VALUES (%s, %s, %s, %s, %s)''', instancia_consulta)


conn.commit()

Agora, com todas as tabelas criadas e preenchidas, podemo encerrar a conexão com a base de dados.

In [ ]:
cursor.close()

conn.close()